這份 Notebook 示範用 Whisper 將 Podcast 語音轉文字，然後用 Map-Reduce 法做長文本摘要

In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [ ]:
import requests
import json
from pprint import pp

In [ ]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=4000):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

## 拿 Podcasts 音訊檔案

In [ ]:
!pip install feedparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6048 sha256=643da679ee35e778fd31f667a42421c05a86423dbd9e58d8655c8e8a47e10d0d
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [ ]:
import feedparser
# 數位時代 https://open.firstory.me/user/bnext
podcast_feed_url = "https://open.firstory.me/rss/user/ckkqnneka7mxc0866j53j5ljw"
podcast_feed = feedparser.parse(podcast_feed_url)
print ("The number of podcast entries is ", len(podcast_feed.entries))

The number of podcast entries is  524


In [ ]:
print(podcast_feed.entries[0])

{'title': '塊轉WEB3_60. 加密貨幣始祖！比特幣是怎麼產生的？', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://feed.firstory.me/rss/user/ckkqnneka7mxc0866j53j5ljw', 'value': '塊轉WEB3_60. 加密貨幣始祖！比特幣是怎麼產生的？'}, 'summary': '<p>全世界第一款加密貨幣「比特幣」，在今年誕生滿15週年，比特幣也是現在市值最大、知名度最高的加密貨幣，隨著美國比特幣現貨ETF的通過，讓比特幣再度登上熱搜。 <br /> <br />你知道為什麼比特幣會有價值？以及發明它的人是誰嗎？ <br /> <br />00:31 加密貨幣始祖！一起來認識比特幣 <br />01:20 比特幣從何而來？ <br />04:21 比特幣是如何產生的？ <br />06:54 比特幣由三大部分組成 <br />08:20 比特幣為什麼有價值？ <br />08:57 中本聰到底是誰？ <br />10:54 為什麼很多比特幣，現在都找不到？ <br />12:22 除了比特幣本身，還有哪些衍伸的應用？ <br /> <br />🎬影片版▸<a href="https://www.youtube.com/watch?v=iw350dgZ7zs" target="_blank">https://www.youtube.com/watch?v=iw350dgZ7zs</a> <br />🔥追蹤WEB3+ 最好懂的區塊鏈媒體 <br />Youtube頻道▸<a href="https://www.youtube.com/@web3plus_official" target="_blank">https://www.youtube.com/@web3plus_off...</a> <br />Line社群▸<a href="https://www.youtube.com/redirect?event=video_description&amp;redir_token=QUFFLUhqbXowTjFISF9yRWd3SlZmd2YtTVhfd2gyaWM4QXxBQ3Jtc0tsT1NYWkJs

In [ ]:
# 挑個檔案最小的那筆
episode_length = 999999999
episode_title = ''
for item in podcast_feed.entries[0:10]:
  for link in item.links:
    if (link['type'] == 'audio/mpeg'):
      if int(link.length) < episode_length:
        episode_url = link.href
        episode_length = int(link.length)
        episode_title = item.title

print(episode_title)
!wget -O 'podcast_episode.mp3' {episode_url}

塊轉WEB3_59. 新春特輯：第一屆幣圈（冷）知識王來了！一起來看你會幾題？
--2024-02-21 05:20:37--  https://chrt.fm/track/AD8A17/m.cdn.firstory.me/play.mp3?url=https%3A%2F%2Fd3mww1g1pfq2pt.cloudfront.net%2FRecord%2Fckkqnneka7mxc0866j53j5ljw%2Fclsa84r8c013401w9h3w21yio.mp3%3Fv%3D1707215684501
Resolving chrt.fm (chrt.fm)... 108.156.60.56, 108.156.60.45, 108.156.60.38, ...
Connecting to chrt.fm (chrt.fm)|108.156.60.56|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://m.cdn.firstory.me/play.mp3?url=https%3A%2F%2Fd3mww1g1pfq2pt.cloudfront.net%2FRecord%2Fckkqnneka7mxc0866j53j5ljw%2Fclsa84r8c013401w9h3w21yio.mp3%3Fv%3D1707215684501 [following]
--2024-02-21 05:20:37--  https://m.cdn.firstory.me/play.mp3?url=https%3A%2F%2Fd3mww1g1pfq2pt.cloudfront.net%2FRecord%2Fckkqnneka7mxc0866j53j5ljw%2Fclsa84r8c013401w9h3w21yio.mp3%3Fv%3D1707215684501
Resolving m.cdn.firstory.me (m.cdn.firstory.me)... 34.81.128.146
Connecting to m.cdn.firstory.me (m.cdn.firstory.me)|34.81.128.146|:443... connected.
HTTP r

## Whisper API

https://platform.openai.com/docs/api-reference/audio

* 上傳的檔案大小限制是 25 MB (如果超過，可以換別的 podcast 看看)
* 格式支援 mp3, mp4, mpeg, mpga, m4a, wav, webm



In [ ]:
data = { "model": "whisper-1", "prompt": "Chinese Mandarin Taiwanese including punctuation" } # whisper 也可以有 prompt 指示
f = open('podcast_episode.mp3', 'rb')
files = [("file", ("podcast_episode.mp3", f, "application/octet-stream"))]

headers = { "Authorization": f'Bearer {openai_api_key}' } # 不需要加 "Content-Type": "multipart/form-data" 我猜 request libray 幫你加了
response = requests.post('https://api.openai.com/v1/audio/transcriptions', headers = headers, data = data, files = files )

# 需要數分鐘來跑

In [ ]:
podcast_transcript = response.text
pp(podcast_transcript)

('{\n'
 '  "text": "B嗎? 不是B 這好像那種什麼國小社會選擇題 不是 簡易舉手 A 也不是A 那就是我了 C 對是C 剪尾刀 哈囉大家好我是靖遠 '
 '歡迎來到這個禮拜的快轉Web Street 今天是大年初五 先祝大家新年快樂 好今天很特別 今天現場除了有 大家熟悉的Ray跟元廷之外 '
 '還有七年資歷的這個交易員景奕 歡迎你來 歡迎歡迎 今天除了是大年初五之外 也是我們Web Street Plus 第一屆的這個幣圈知識王 '
 '我們今天會有十題的題目 讓大家搶答 然後最後我們會有小獎品 就可以知道今年 對於幣圈最熟悉的小知識王是誰了 好那我們就先進入到第一部分 好我們先來講一下 '
 '第一階段的這個遊戲規則 第一階段是搶答題 我們會有 每個題目會有四個答案 然後大家如果知道答案的話 就是先舉手上的牌子搶答 然後最先舉的可以先回答 '
 '然後我們每答對一題就可以有一分 然後最後我們來算一下 大家總分是多少 好那我們現在先進入到第一題 第一題看似簡單 可是其實蠻難的 台灣現在還有在營運的 '
 '叫本土交易所 第一家成立的是哪一家 那A是幣托 B是mycoin 然後C是X D是Xrex 好請搶答 元廷 我覺得是mycoin 不是 那還有機會 '
 '還可以再答嗎 菁英 接著搶 等一下都接著搶嘛 對 好 幣托 對是幣托 其實他們兩個都是2014年成立的 然後只是他們的月份不太一樣 幣托是2014年3月 '
 '然後mycoin是5月 所以對差了兩個月的時間 嗯 好 我本來也猜mycoin 對 大家會以為mycoin是比較早期的 那其實因為他們同一年成立嘛 '
 '所以在今年他們都10歲了 迎來10週年 非常不容易 10年的時間過得非常快 好 然後進入到第二題 第二題是大家非常熟悉的 這個比特幣披薩日 '
 '到底是幾月幾號 然後因為如果只有 幾月幾號有點太簡單 所以我們前面再加上了年份 A是2010年5月23 B是2011年5月22日 '
 'C是2010年5月22日 D是2011年5月23日 等一下你還沒 喔 C C 對沒錯這是有2010年的5月22日 然後就是在這個時間點呢 有一名叫做 '
 '這個哈涅茨的工程師 他花了一萬克的比特幣 買了兩片的披薩 然後這個交易被普遍被認為是 比特幣的第一筆商業交易 所以每一年的5月22日 '


## 算一下有多少 token

In [ ]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.6 MB/s eta 0:00:00


In [ ]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")
num_tokens = len(encoding.encode(podcast_transcript))
print(num_tokens)

4074


# 直接摘要

此 prompt 出處: https://platform.openai.com/docs/tutorials/meeting-minutes

In [ ]:
messages = [{
        "role": "system",
        "content": "(zh-tw) You are a highly skilled AI trained in language comprehension and summarization. I would like you to read the following text and summarize it into a concise abstract paragraph. Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text. Please avoid unnecessary details or tangential points."
    },
    {
        "role": "user",
        "content": podcast_transcript
    }
]

abstract_summary = get_completion(messages)
pp(abstract_summary)

'在這段討論中，主持人介紹了一個幣圈知識競賽活動，邀請了參賽者回答與加密貨幣相關的問題。比賽包括多個問題，從台灣本土交易所的成立到比特幣披薩日的日期，以及比特幣價格的換算等。此外，也提到了台灣第一檔STO的名稱以及LINE的NFT平台名稱。最後，參賽者回答了有關全球最大交易所新執行長的中文名字以及以太坊現貨ETF的申請機構數量等問題。最終，一位參賽者成為了幣圈知識王。整體而言，這是一場關於加密貨幣相關知識的競賽活動。'


換個摘要 prompt 輸出條列也不錯:

In [ ]:
messages = [{
        "role": "system",
        "content": "(zh-tw) You are a proficient AI with a specialty in distilling information into key points. Based on the following text, identify and list the main points that were discussed or brought up. These should be the most important ideas, findings, or topics that are crucial to the essence of the discussion. Your goal is to provide a list that someone could read to quickly understand what was talked about."
    },
    {
        "role": "user",
        "content": podcast_transcript
    }
]

key_points = get_completion(messages, model = "gpt-3.5-turbo-16k")
print(key_points)

- 第一部分是幣圈知識王的搶答遊戲
- 第一題是台灣第一家成立的本土交易所是幣托
- 第二題是比特幣披薩日是2010年5月22日
- 第三題是持有一顆比特幣以上的錢包數量已突破100萬
- 第四題是今天比特幣的價格是128萬新台幣
- 第五題是台灣第一檔STO的全名是陽光綠意債務型STO
- 第二部分是用小白板作答的遊戲
- 第一題是全球最大交易所必要化的新執行長的中文名字是鄧偉政
- 第二題是Solana的創辦人雅柯文柯是烏克蘭人
- 第三題是LINE NFT平台的全名是LINE Dolce
- 第三部分是以太坊現貨ETF申請的機構總共有7家
- 仁廷是第一屆的幣圈知識王


## 來挑戰 1 小時的 podcast (超過25MB怎麼辦????)

In [ ]:
# 塞掐 Side Chat (來聽fox大大!!)
# https://omny.fm/shows/side-chat/playlists/side-chat
podcast_feed_url = "https://www.omnycontent.com/d/playlist/a4cc0a4a-642d-45d7-ac5d-ac5600c620b0/63669eef-a99a-40f9-9112-acb400c15ad3/44310870-6c8c-4b11-b482-acb400c1832d/podcast.rss"
podcast_feed = feedparser.parse(podcast_feed_url)
print ("The number of podcast entries is ", len(podcast_feed.entries))

The number of podcast entries is  100


In [ ]:
for item in podcast_feed.entries[0].links:
  if (item['type'] == 'audio/mpeg'):
    episode_url = item.href
!wget -O 'long_podcast_episode.mp3' {episode_url}

--2024-02-21 05:22:11--  https://track.fstry.me/p/u6sj5e6d/sw.soundon.fm/p/2CWTSS/traffic.omny.fm/d/clips/a4cc0a4a-642d-45d7-ac5d-ac5600c620b0/63669eef-a99a-40f9-9112-acb400c15ad3/bb523e3e-8acd-4d15-901c-b1190031a7d9/audio.mp3?utm_source=Podcast
Resolving track.fstry.me (track.fstry.me)... 35.229.184.97
Connecting to track.fstry.me (track.fstry.me)|35.229.184.97|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://sw.soundon.fm/p/2CWTSS/traffic.omny.fm/d/clips/a4cc0a4a-642d-45d7-ac5d-ac5600c620b0/63669eef-a99a-40f9-9112-acb400c15ad3/bb523e3e-8acd-4d15-901c-b1190031a7d9/audio.mp3?utm_source=Podcast [following]
--2024-02-21 05:22:12--  https://sw.soundon.fm/p/2CWTSS/traffic.omny.fm/d/clips/a4cc0a4a-642d-45d7-ac5d-ac5600c620b0/63669eef-a99a-40f9-9112-acb400c15ad3/bb523e3e-8acd-4d15-901c-b1190031a7d9/audio.mp3?utm_source=Podcast
Resolving sw.soundon.fm (sw.soundon.fm)... 34.81.110.127
Connecting to sw.soundon.fm (sw.soundon.fm)|34.81.110.127|:443... connec

### 嘗試用 OpenAI API

In [ ]:
data = { "model": "whisper-1", "prompt": "請用台灣繁體中文" } #  flac, mp3, mp4, mpeg, mpga, m4a, ogg, wav, or webm
f = open('long_podcast_episode.mp3', 'rb')
files = [("file", ("long_podcast_episode.mp3", f, "application/octet-stream"))]

headers = { "Authorization": f'Bearer {openai_api_key}' } # 不需要加 "Content-Type": "multipart/form-data" 因為我猜 request libray 幫你加了
response = requests.post('https://api.openai.com/v1/audio/transcriptions', headers = headers, data = data, files = files )
pp(response.text)

('{\n'
 '  "error": {\n'
 '    "message": "Maximum content size limit (26214400) exceeded (26280494 '
 'bytes read)",\n'
 '    "type": "server_error",\n'
 '    "param": null,\n'
 '    "code": null\n'
 '  }\n'
 '}')


OpenAI API 至多 25M，狠狠超過了.....
那就來自己 hosting model 吧!! 官方有釋出模型可以自行安裝。

* 官方版 https://github.com/openai/whisper
* CPU 效能優化版 https://github.com/ggerganov/whisper.cpp

在這個 colab 跑的話，記得選 GPU（編輯 -> 筆記本設定 -> 硬體加速器: 選個 GPU）

In [ ]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 11.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=382dca5f35435da1ed0208a93cb057cd7987fc0c7b38d58a7df0610bacad1509
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper


In [ ]:
  import whisper
  print ("Download and Load the Whisper model")
  model = whisper.load_model("medium") # 模型有分不同大小 medium 1.42G, large: 2.87G
  print (model.device) # cpu 會跑很久..... 因此一開始要選 gpu

Download and Load the Whisper model


100%|██████████████████████████████████████| 1.42G/1.42G [00:13<00:00, 113MiB/s]


cuda:0


In [ ]:
result = model.transcribe("long_podcast_episode.mp3") # 一小時長度，用 T4 GPU 跑需要約 20 分鐘

In [ ]:
long_podcast_transcript = result["text"]
pp(long_podcast_transcript)

('- 嗨,大家欢迎回到Sidecar,我是Inside的Joss- 今天Mia不在,所以是Joss带班,我是Inside的Fox- '
 '那今天Sidecar很开心邀请到的是这个精诚资讯的Stanley- Stanley协力,今天跟我们聊一下这个产品叫One EC- '
 '请Stanley自我介绍一下- Hi Fox,各位听众大家好,我是Morgan Stanley的Stanley- 但是我们不是做金融的,我是做电商- '
 '服务于精诚资讯的One EC,那负责BD战略伙伴的建立- 从2006年一直到现在都一直投入在电商产业- 你从2006年就开始做电商了- '
 '小时候的时候就在Momo服务- 以前在Momo待多久啊?- 有一段时间,我们从一开始电视购物网路购物的早期的时候- 我是算最早期的- '
 '你从电视购物就在了- 所以电视购物到...- 我觉得Momo...- 但是Momo还小啦,还在亚户跟PC用的后面- '
 'OK,所以那时候大家都想说这电视购物可能跟这种线上的没什么关系- 当然,因为那时候我们就要你好,我是Momo- 他就说电视购物,东生嘛- 我们没有兴趣- '
 '真的啊?- 对,因为那时候Momo,PC home亚户大家都知道- 但是Momo他才停留在电视购物跟东生的那个时间- '
 '欸,我借这个机会,虽然我们今天主题是One Easy- 但是我觉得这整个过程的,我觉得是蛮有趣的- 比如说那时候你在里面看到的就是说- '
 '那怎么样突然间想到说- 我们应该从电视购物慢慢转型到电子商务- 然后甚至其实后来在电子商务上一直开始慢慢的开始超前领先的这一段- 你有参与到吗?- '
 '我大概分享一下,其实以电视购物到电商的这一块- Momo当初的布局是从电视购物、电商、行路购物- 甚至那时候线下的百货跟药妆都有做布局- '
 '因为也想做多方的尝试- 那我们再看从电视购物到电商的这一块- 其实电视购物因为他的收视群就是婆婆妈妈- 还有一些不用上班在家可以看电视购物的这些人- '
 '所以他的受众族群比较扁平就是这一个受众族群而已- 那个时间点在2006那个时间点- 网络购物刚开始就是从Yahoo、PC '
 'Home、Ebay、Amazon蓬勃在起飞- 所以这个是一个很新的一块市场- 所以我们那时候投入在电商这一块我们看到的是- '
 

In [ ]:
long_num_tokens = len(encoding.encode(long_podcast_transcript))
print(long_num_tokens)

16583


 這種長文本，一個 prompt 塞不下 16k 的 gpt-3.5，要怎麼做摘要?

1. 換 context window 更大的模型，例如 gpt-4-turbo 可以 128k、gemini-pro 1.5 可以 1000k
2. 將內容拆段處理

## Map-Reduce 方法

這裡我們直接拿 langchain 這個套件的拆段落方法來用

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

langchain 的 RecursiveCharacterTextSplitter 方式是個常見的拆段落方式

https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter

預設會依序用這四個字元來拆 ["\n\n", "\n", " ", ""]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

texts = text_splitter.split_text(long_podcast_transcript)
len(texts)

9

不同程式語言處理平行非同步很不一樣，這裡用 threading....
我在 Ruby on Rails 專案中，則是用 background jobs 放到 queue 來平行處理，這樣也就有 retry 機制

In [ ]:
import threading

In [ ]:
def process_summary(index, content, results):
  # Tl;dr 是 too long; didn't read 的意思，國外鄉民常用的懶人包縮寫
  messages = [
    {
      "role": "user",
      "content": content + ' Tl;dr (#zh-tw):'
    }
  ]
  print("Sent request: " + content)
  abstract_summary = get_completion(messages, model="gpt-3.5-turbo-16k")
  results[index] = abstract_summary

results = [None] * len(texts)
threads = []
# 透過 threads 平行發送所有 requests 進行摘要
for i, item in enumerate(texts):
    t = threading.Thread(target=process_summary, args=(i, item, results))
    t.start()
    threads.append(t)

for t in threads:
    t.join()  # 等待每個 thread 都完成


print(results)

Sent request: - 嗨,大家欢迎回到Sidecar,我是Inside的Joss- 今天Mia不在,所以是Joss带班,我是Inside的Fox- 那今天Sidecar很开心邀请到的是这个精诚资讯的Stanley- Stanley协力,今天跟我们聊一下这个产品叫One EC- 请Stanley自我介绍一下- Hi Fox,各位听众大家好,我是Morgan Stanley的Stanley- 但是我们不是做金融的,我是做电商- 服务于精诚资讯的One EC,那负责BD战略伙伴的建立- 从2006年一直到现在都一直投入在电商产业- 你从2006年就开始做电商了- 小时候的时候就在Momo服务- 以前在Momo待多久啊?- 有一段时间,我们从一开始电视购物网路购物的早期的时候- 我是算最早期的- 你从电视购物就在了- 所以电视购物到...- 我觉得Momo...- 但是Momo还小啦,还在亚户跟PC用的后面- OK,所以那时候大家都想说这电视购物可能跟这种线上的没什么关系- 当然,因为那时候我们就要你好,我是Momo- 他就说电视购物,东生嘛- 我们没有兴趣- 真的啊?- 对,因为那时候Momo,PC home亚户大家都知道- 但是Momo他才停留在电视购物跟东生的那个时间- 欸,我借这个机会,虽然我们今天主题是One Easy- 但是我觉得这整个过程的,我觉得是蛮有趣的- 比如说那时候你在里面看到的就是说- 那怎么样突然间想到说- 我们应该从电视购物慢慢转型到电子商务- 然后甚至其实后来在电子商务上一直开始慢慢的开始超前领先的这一段- 你有参与到吗?- 我大概分享一下,其实以电视购物到电商的这一块- Momo当初的布局是从电视购物、电商、行路购物- 甚至那时候线下的百货跟药妆都有做布局- 因为也想做多方的尝试- 那我们再看从电视购物到电商的这一块- 其实电视购物因为他的收视群就是婆婆妈妈- 还有一些不用上班在家可以看电视购物的这些人- 所以他的受众族群比较扁平就是这一个受众族群而已- 那个时间点在2006那个时间点- 网络购物刚开始就是从Yahoo、PC Home、Ebay、Amazon蓬勃在起飞- 所以这个是一个很新的一块市场- 所以我们那时候投入在电商这一块我们看到的是- 接下来的成长曲线会是一个很漂亮的成长曲线- 那电视购物应该在两三年后就大概是爆合了- 

In [ ]:
# 把所有子摘要集合起來
content = "\n".join("\n" + item for item in results)
messages = [
    {
      "role": "user",
      "content": content + ' Tl;dr (#zh-tw):'
    }
]

print(content)


今天的节目邀请到了精诚资讯的Stanley来介绍他们的产品One EC。Stanley在2006年开始从事电商行业，他曾在Momo服务，并参与了从电视购物到电子商务的转型过程。他认为Momo之所以成功转型，是因为抓住了风口和赛道，并进行了多方面的投资。他还提到Momo曾尝试过百货和药妆业务，但最终卖给了其他公司。在Momo的早期，他们主要做B2C电商，后来才开始涉足B2B2C和购物商城。Stanley在2011年离开了Momo，转到了B2B和B2B2C的服务领域。

这段对话中，讨论了京城资讯（Jingcheng Information）在电商领域的发展。最早，京城资讯只做B2C业务，后来也开始涉足购物商城等领域。在2011年左右，对电商的发展趋势有了更多的认识后，京城资讯开始转向B2B和B2B2C服务，主要面向供应商。从2020年开始，京城资讯开始关注电商领域，并决定建立一个团队来开展相关服务。他们选择切入电商领域的原因是因为电商领域有很多可以做的事情，而且他们的团队成员都来自电商行业。他们的目标是通过提供一站式的解决方案，帮助商家管理订单、销售数据等，并将所有的电商业务集中到一个平台上，这就是他们所称的万EC（EC all in one）的由来。总之，京城资讯希望通过提供综合的电商解决方案，为客户创造价值。

传统的做法是在每个平台后台登录并进行上下架操作，非常麻烦。此外，如果有多个平台，订单就需要到处处理。为了解决这个问题，可以自己雇人编写一个程序来管理所有平台的订单和相关数据，包括商品上下架和刊登等。然而，这样的做法只有头部企业才能实现，中小型企业很难做到。因此，京城希望能够帮助他们解决这个问题，实现电商全通路的营运整合，包括订单管理和销售数据分析等。京城支持台湾的主要开店平台，如九一APP、7bit、sharp line、sharp file、EasyStore等。他们的目标是帮助商家解决营收和获利的问题，而不是像传统的库存管理一样。京城认为最困难的是将多个平台的订单和与钱有关的事务整合在一起，以及销售数据的管理。他们的收费方式根据使用量计费，相比于雇佣人力成本来说非常划算。他们将传统的人工智慧转变为人工智慧，通过API和资讯串流实现及时的数据管理。

这段对话中，讨论了使用整合系统的好处和挑战。使用整合系统可以提供及时准确的数据，而不是依赖手动整理报表

In [ ]:
# 最後將上述加總的小摘要，再摘要一次
final_summary = get_completion(messages, model="gpt-4-turbo-preview")

In [ ]:
print(final_summary)

Stanley从2006年开始从事电商行业，曾在Momo工作并参与其从电视购物到电子商务的转型。他在2011年离开Momo，转向B2B和B2B2C服务领域。京城资讯（Jingcheng Information）最初只做B2C业务，后来转向B2B和B2B2C服务，主要面向供应商。从2020年开始，京城资讯决定建立一个团队来开展电商服务，目标是提供一站式解决方案，帮助商家管理订单、销售数据等，通过API和资讯串流实现及时的数据管理。他们支持台湾的主要开店平台，旨在帮助商家解决营收和获利问题。使用整合系统可以节省时间和人力成本，提供智能化功能，如自动配料。客户反馈表示，使用整合系统后，原本需要5-6个人的工作现在只需1.5个人就能完成。此外，还讨论了理财话题，强调稳中求进、专注价值和持续进化的重要性，以及远离负能量和学习新知识的重要性。
